In [13]:
%matplotlib notebook
import sys
sys.path.append("/home/stachu/Projects/Anomaly_detection/Forecasting_models")

from tqdm.auto import tqdm
import torch
from torch.nn.functional import mse_loss
import plotly.graph_objects as go

from predpy.experimentator import Experimentator, load_experimentator
from predpy.dataset import MultiTimeSeriesDataloader
from predpy.plotter.plotter import plot_3d_embeddings
from tsad.noiser import apply_noise_on_dataframes, white_noise

In [2]:
exp = load_experimentator("../saved_experiments/2022-01-06_21:33:23.pkl")
model = exp.load_pl_model(
    model_idx=0,
    dir_path="../checkpoints/household_power_consumption/LSTMVAE_h200_l1",
    # file_name="2022-01-06_19:51:39.ckpt",
    # dir_path="./checkpoints",
    # file_name="epoch=0-step=23542",
    find_last=True
)

In [3]:
tsm = exp.load_time_series_module(0)

normal_dfs = tsm.get_data_from_range(start=-10000, end=-3000, copy=True)
anomaly_dfs = tsm.get_data_from_range(start=-3000, copy=True)

apply_noise_on_dataframes(
    anomaly_dfs, make_noise=white_noise, negativity="abs", loc=1, scale=0.35)
pass

In [4]:
normal_dl = MultiTimeSeriesDataloader(
    tsm.get_data_from_range(start=-10000, end=-3000, copy=True),
    tsm.window_size, tsm.target, batch_size=tsm.batch_size)
dataloaders = [normal_dl]

loc_scale_pairs = [
    (0, 0.02), (0, 0.05), (0, 0.07), (0, 0.2), (0, 0.4), (0, 0.5),
    (0.2, 0.2), (0.2, 0.4), (0.3, 0.1), (0.5, 0.2)]

for loc, scale in loc_scale_pairs:
    dataloaders += [MultiTimeSeriesDataloader(
        apply_noise_on_dataframes(
            tsm.get_data_from_range(start=-10000, end=-3000, copy=True),
            make_noise=white_noise, negativity="abs", loc=1, scale=0.35),
        tsm.window_size, tsm.target, batch_size=tsm.batch_size)]

In [18]:
zs, z_distances, recon_errs = [], [], []
for dataloader in dataloaders:
    for batch in tqdm(dataloader):
        input_, _ = model.get_Xy(batch)
        z = model.encode(input_)
        rec = model.decode(z)
        for i in range(len(z)):
            zs += [z[i].tolist()]
            z_distances += [torch.norm(z[i]).item()]
            recon_errs += [mse_loss(input_[i], rec[i]).item()]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

In [31]:
from scipy.stats import pearsonr, spearmanr

print("Pearson %.3f, p-val %.2f" % pearsonr(z_distances, recon_errs))
print("Spearman %.3f, p-val %.2f" % spearmanr(z_distances, recon_errs))

data = []
start, end = 0, 0
for i, dl in enumerate(dataloaders):
    start = end
    end += len(dl)
    if i == 0:
        name = "normal data"
    else:
        name = str(loc_scale_pairs[i-1])
    data += [
        go.Scatter(
            x=z_distances[start:end],
            y=recon_errs[start:end],
            mode='markers', name=name,
        )
    ]

layout = go.Layout(
    title="Z norms and reconstruction errors",
    yaxis=dict(title='reconstruction error'),
    xaxis=dict(title='z norm')
)

fig = go.Figure(data=data, layout=layout)
fig.show()

Pearson 0.901, p-val 0.00
Spearman 0.450, p-val 0.00


In [24]:
from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components=3)
embs_3d = pca.fit_transform(np.array(zs)[:, 0, :])

In [25]:
data = []
start, end = 0, 0
for i, dl in enumerate(dataloaders):
    start = end
    end += len(dl)
    if i == 0:
        name = "normal data"
    else:
        name = str(loc_scale_pairs[i-1])
    data += [
        go.Scatter3d(
            x=embs_3d[start:end, 0].tolist(),
            y=embs_3d[start:end, 1].tolist(),
            z=embs_3d[start:end, 2].tolist(),
            mode='markers', name=name,
        )
    ]

layout = go.Layout(
    title="VAE embeddings with PCA dimentions reduction",
)

fig = go.Figure(data=data, layout=layout)
fig.show()